# Formating the data

### Imports
- tweepy - The data object that the data is saved in
- pickle - To unpickle the variables that have been saved
- pandas - To reprosent the data in a dataframe
- numpy - Working with arrays
- glob - Easy way to gather all the files in a folder

In [3]:
import tweepy
import pickle
import pandas
import numpy as np
import glob

ModuleNotFoundError: No module named 'tweepy'

## Testing the data

This is just to understand that data in each file and how to index it.
to get the information that we want we need to index the key firstly, then there will be a list of tweet data objects. To get the text of the tweet its then just ```.text```

The data comes in as such:
```{"user" : [{tweet}, {tweet}, {tweet}]}```

In [ ]:
#file = "../tweets/12.p"

In [ ]:
#with open (file, "rb") as f:
#    data = pickle.load(f)
#    f.close()
#print(data)

In [ ]:
#data[12][0].text

## Collecting the data

This chunk of code collects all the names of the files that need to be opened so they can be user with the ```open()``` function to read all of the data in each file. 

The next chunk of code is similar to above where it collects all of the informaiont of the users tweets and adds them to a dictionary with an array of users and an array of users 1st, 2nd, ..., 1000th tweet. This seemed to be the easiest way to format the data to work with pandas data frame and show the data cohesivley. The data also needs to have the same amount of tweets for each user so we have to check if they have enough tweets before we start putting thier informaion in the data.

The last part of the code trys to append the tweet text to the tweet array, however, if it is the first tweet to go in then it has to create that array so the exception does so.

Finally we can add this dictionary to the data frame and show it in a nice looking table.

In [ ]:
files = []
for name in glob.glob('../tweets/*'):
    print (name)
    files.append(name)

In [1]:
userData = {
    "User": np.array([])
}
for file in files:
    with open(file, 'rb') as f:
        accounts = pickle.load(f)
        for user, value in accounts.items():
            if (len(value) < 1000):
                break
            userData["User"]=np.append(userData["User"], [user])
            for i in range(1000):
                tweet = value[i]
                try:
                    userData[i] = np.append(userData[i], [tweet.text])
                except:
                    userData[i] = np.array([tweet.text])
            

NameError: name 'np' is not defined

In [ ]:
df = pandas.DataFrame(userData)
df.style